In [5]:
import networkx as nx
import pandas as pd

Загружаю ксль файл 40 минут

In [7]:
dataset = pd.read_excel('globalterrorismdb_0718dist.xlsx')

Выделяю нужные мне айдишники

In [232]:
g1 = []
g2 = []
for i in range(len(dataset['eventid'])):
    if(dataset['iyear'][i] == 2007):
        g1.append([dataset['iyear'][i], dataset['eventid'][i], dataset['related'][i]])
    if(dataset['iyear'][i] == 2011):
        g2.append([dataset['iyear'][i], dataset['eventid'][i], dataset['related'][i]])
print(len(g1), len(g2))

3242 5076


Составляю начальные графы

In [233]:
graph1 = nx.Graph()
for i in g1:
    if(str(i[2])!= 'nan'):
        for j in i[2].split(', '):
            if(i[1] != int(j) and i[1] < int(j)):
                graph1.add_edge(i[1], int(j))

In [234]:
graph2 = nx.Graph()
for i in g2:
    if(str(i[2])!= 'nan'):
        for j in i[2].split(', '):
            if(i[1] != int(j) and i[1] < int(j)):
                graph2.add_edge(i[1], int(j))


Выделяю компоненты связанности в этих графах

In [235]:
vis1 = dict()
vis2 = dict()
for i in graph1.nodes:
    vis1[i] = 0
for i in graph2.nodes:
    vis2[i] = 0

In [236]:
def dfs1(i, c):
    global vis1
    vis1[i] = c
    for j in graph1.edges(i):
        if(vis1[j[1]] == 0):
            dfs1(j[1], c)

In [237]:
col1 = 0
for i in graph1.nodes:
    if(vis1[i] == 0):
        col1 += 1
        dfs1(i, col1)

In [238]:
def dfs2(i, c):
    global vis2
    vis2[i] = c
    for j in graph2.edges(i):
        if(vis2[j[1]] == 0):
            dfs2(j[1], c)

In [239]:
col2 = 0
for i in graph2.nodes:
    if(vis2[i] == 0):
        col2 += 1
        dfs2(i, col2)

In [240]:
comps1 = [0] * (col1 + 1)
comps2 = [0] * (col2 + 1)
for i in vis1:
    comps1[vis1[i]] += 1
for i in vis2:
    comps2[vis2[i]] += 1

Выделяю компоненты максимального размера

In [241]:
for i in range(len(comps1)):
    comps1[i] = [comps1[i], i]
for i in range(len(comps2)):
    comps2[i] = [comps2[i], i]
comps1.sort()
comps2.sort()

In [242]:
verts1 = dict()
verts2 = dict()
compcol1 = []
compcol2 = []
for i in range(-1, -11, -1):
    compcol1.append(comps1[i][1])
    compcol2.append(comps2[i][1])
    verts1[comps1[i][1]] = 0
    verts2[comps2[i][1]] = 0

Тут я связываю 10 самых больших компонент в граф самым простым способом: последовательно. 
Однако после я изменил этот способ. Я их стал связывать в дерево клик(то есть для 10 клик добавляю 9 ребер чтобы граф стал
связанным). Мосты я выпускал из одной и той же вершины для каждой клики. Это можно расширить выпуская из случайной вершины
клики. Это немного усложнит вычисление GED. Однако остальные алгоритмы не пострадают.

In [243]:
workgra1 = nx.Graph()
for i in vis1:
    if(vis1[i] in compcol1):
        verts1[vis1[i]] = i
        workgra1.add_node(i)
workgra2 = nx.Graph()
for i in vis2:
    if(vis2[i] in compcol2):
        verts2[vis2[i]] = i
        workgra2.add_node(i)

In [109]:
for i in workgra1.nodes:
    for j in graph1.edges(i):
        if(j[1] > i):
            workgra1.add_edge(i, j[1])
for i in range(9):
    workgra1.add_edge(list(verts1.items())[i][1], list(verts1.items())[i + 1][1])
    
for i in workgra2.nodes:
    for j in graph2.edges(i):
        if(j[1] > i):
            workgra2.add_edge(i, j[1])
for i in range(9):
    workgra2.add_edge(list(verts2.items())[i][1], list(verts2.items())[i + 1][1])

Тут я преобразовываю матрицу смежностей. В следующих ячейках я выполняю алгоритм флойда.

In [246]:
amat1 = nx.adj_matrix(workgra1)
ammat1 = []
for i in range(len(workgra1.nodes)):
    ammat1.append([])
    for j in range(len(workgra1.nodes)):
        ammat1[i].append(amat1[i, j])
        if(ammat1[i][j] == 0):
            ammat1[i][j] = 1000

amat2 = nx.adj_matrix(workgra2)
ammat2 = []
for i in range(len(workgra2.nodes)):
    ammat2.append([])
    for j in range(len(workgra2.nodes)):
        ammat2[i].append(amat2[i, j])
        if(ammat2[i][j] == 0):
            ammat2[i][j] = 1000


In [250]:
for i in range(len(ammat1)):
    for j in range(len(ammat1)):
        for k in range(len(ammat1)):
            if(not (i == j and ammat1[i][k] == 1)):
                ammat1[i][j] = min(ammat1[i][j], ammat1[i][k] + ammat1[k][j])
                ammat1[j][i] = min(ammat1[j][i], ammat1[i][k] + ammat1[k][j])
                if(ammat1[i][j] != ammat1[j][i]):
                    print('wat')

In [251]:
for i in range(len(ammat2)):
    for j in range(len(ammat2)):
        for k in range(len(ammat2)):
            if(not (i == j and ammat2[i][k] == 1)):
                ammat2[i][j] = min(ammat2[i][j], ammat2[i][k] + ammat2[k][j])
                ammat2[j][i] = min(ammat2[j][i], ammat2[i][k] + ammat2[k][j])
                if(ammat2[i][j] != ammat2[j][i]):
                    print('wat')

Тут я получаю Диаметр для графа и eccentricity. Я вывожу две строчки для двух графов. В каждой строчке первой идет Диаметр
затем идет отстортированный список eccentricity

In [252]:
diam1 = max(ammat1[0])
eccentricity1 = []
for i in ammat1:
    diam1 = max(diam1, max(i))
    eccentricity1.append(max(i))

eccentricity1.sort()

diam2 = max(ammat2[0])
eccentricity2 = []
for i in ammat2:
    diam2 = max(diam2, max(i))
    eccentricity2.append(max(i))

eccentricity2.sort()

print(diam1, ' '.join(map(str, eccentricity1)))
print(diam2, ' '.join(map(str, eccentricity2)))

8 4 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 8 8 8
8 4 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8


Тут я вывожу periphery для обеих графов

In [253]:
per1 = 0
for i in eccentricity1:
    if(i == diam1):
        per1 += 1

per2 = 0
for i in eccentricity2:
    if(i == diam2):
        per2 += 1       

print(per1, per2)

14 15


Тут я честно считаю минимальные циклы для графов. Это малоосмысленно тк в каждой клике найдется оч много циклов
размера 3.

In [254]:
stak = []
ans = 1000
vis1 = dict()

def dfs(i):
    if(ans == 3):
        return
    global ans
    global stak
    global vis1
    vis1[i] = True
    stak.append(i)
    if(len(stak) >= 3):
        for j in workgra1.edges(i):
            if(j[1] in stak[:len(stak) - 2]):
                ans = min(ans, len(stak) - stak.index(j[1]))
                
    for j in workgra1.edges(i):
        if(not j[1] in vis1.keys()):
            dfs(j[1])
            if(ans == 3):
                return
    stak.remove(i)

dfs(list(workgra1.nodes())[0])
print(ans)

3


<ipython-input-254-e8a297920867>:8: SyntaxWarning: name 'ans' is used prior to global declaration
  global ans


In [255]:
stak = []
ans = 1000
vis1 = dict()

def dfs(i):
    if(ans == 3):
        return
    global ans
    global stak
    global vis1
    vis1[i] = True
    stak.append(i)
    if(len(stak) >= 3):
        for j in workgra2.edges(i):
            if(j[1] in stak[:len(stak) - 2]):
                ans = min(ans, len(stak) - stak.index(j[1]))
                
    for j in workgra2.edges(i):
        if(not j[1] in vis1.keys()):
            dfs(j[1])
            if(ans == 3):
                return
    stak.remove(i)

dfs(list(workgra2.nodes())[0])
print(ans)

3


<ipython-input-255-051c976ccf98>:8: SyntaxWarning: name 'ans' is used prior to global declaration
  global ans


Тут более сложное создание графов, о котором я писал выше.

In [244]:
workgra1 = nx.Graph()
for i in vis1:
    if(vis1[i] in compcol1):
        verts1[vis1[i]] = i
        workgra1.add_node(i)
workgra2 = nx.Graph()
for i in vis2:
    if(vis2[i] in compcol2):
        verts2[vis2[i]] = i
        workgra2.add_node(i)

In [245]:
for i in workgra1.nodes:
    for j in graph1.edges(i):
        if(j[1] > i):
            workgra1.add_edge(i, j[1])
added = [compcol1[0]]
notin = compcol1[1:]
compgra1 = nx.Graph()
for i in compcol1:
    compgra1.add_node(i)

from random import randint
for i in range(9):
    a = randint(0, i)
    b = randint(0, 9 - i - 1)
    compgra1.add_edge(added[a], notin[b])
    workgra1.add_edge(verts1[added[a]], verts1[notin[b]])
    added.append(notin[b])
    notin.remove(notin[b])
    
for i in workgra2.nodes:
    for j in graph2.edges(i):
        if(j[1] > i):
            workgra2.add_edge(i, j[1])
added = [compcol2[0]]
notin = compcol2[1:]
compgra2 = nx.Graph()
for i in compcol2:
    compgra2.add_node(i)

from random import randint
for i in range(9):
    a = randint(0, i)
    b = randint(0, 9 - i - 1)
    compgra2.add_edge(added[a], notin[b])
    workgra2.add_edge(verts2[added[a]], verts2[notin[b]])
    added.append(notin[b])
    notin.remove(notin[b])

Тут считается GED. Он работает примерно час. Работает он учитывая особенности графа а именно то что оба графа являются
кликами соединенными мостами. То есть "дерево" клик. Так как оба графа имеют абсолютно разные именования вершин, то я
не учитывал переименование вершин как операцию. Так как оба графа имеют примерно равные названия клик + эти клики
имеют довольно большие размеры, то я предполагаю, что разумнее сопоставлять графы не по вершинам, а по кликам.
То есть первый граф имеет 10 клик и второй имеет 10 клик, первый имеет 9 доп ребер и второй. Следовательно я просто иду
по всем возможным переставлениям клик. Затем я иду по каждой паре сопоставленных клик, считаю добавление вершин и ребер.
Затем для каждой клики я считаю Ребра из нее в другие клики которые есть в одном графе, но отсутствуют в другом.
Я вычисляю минимум из таких расстояний. Для графа, где из одной клики мосты в другие клики могут выходить из разных вершин
считаются еще переставление одного конца ребра внутри клики.

In [313]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
ans = 100000000
for i in range(3628800):
    eq = dict()
    eq1 = dict()
    res = 0
    for j in range(10):
        eq[comps1[j - 10][1]] = comps2[a[j] - 11][1]
        eq1[comps2[a[j] - 11][1]] = comps1[j - 10][1]
        res += abs(comps1[j - 10][0] - comps2[a[j] - 11][0])
        res += abs(comps1[j - 10][0] * (comps1[j - 10][0] - 1) // 2 - comps2[a[j] - 11][0] * (comps2[a[j] - 11][0] - 1) // 2)
    for j in compgra1.edges:
        if(not ((eq[j[0]], eq[j[1]]) in compgra2.edges)):
            res += 1
    for j in compgra2.edges:
        if(not ((eq1[j[0]], eq1[j[1]]) in compgra1.edges)):
            res += 1
        
    
    
    ans = min(ans, res)
    next_permutation(a)

print(ans)

121


Wiener_index для двух графов считаем из матрицы полученной после Флойда

In [292]:
weiner = 0
for i in range(len(ammat1)):
    for j in range(i + 1, len(ammat1)):
        weiner += ammat1[i][j]
print(weiner)


weiner = 0
for i in range(len(ammat2)):
    for j in range(i + 1, len(ammat2)):
        weiner += ammat2[i][j]
print(weiner)

10947
11597


Hosoya index также учитывает что граф это дерево клик. Я перебираю наборы ребер соединяющих клики. Добавив очередное ребро
я считаю hosoya index для каждой клики(предпосчитано) перемножаю и добавляю к ответу

In [323]:
HOS1 = 0
hm = [1, 1, 2, 4, 10, 26, 76, 232, 764, 2620, 9496, 35696, 140152, 568504, 2390480, 10349536, 46206736, 211799312, 997313824, 4809701440]
cscscscs1 = comps1[len(comps1) - 10:len(comps1)]
cscscs1 = dict()
for i in cscscscs1:
    cscscs1[i[1]] = i[0]

for i in range(2 ** 9):
    preres = 0
    tst = []
    j1 = 0
    fl = True
    for j in compgra1.edges:
        if((i // (2 ** j1)) % 2 == 1):
            if(j[0] in tst or j[1] in tst):
                fl = False
                break
            tst.append(j[0])
            tst.append(j[1])
            preres += 1
        j1 += 1
    if(fl):
        pass
    else:
        preres = 1
        for i in tst:
            cscscs1[i] -= 1
        for i in cscscs1:
            preres *= hm[cscscs1[i]]
        for i in tst:
            cscscs1[i] += 1
        HOS1 += preres
HOS1

645820541952923168914014208

In [324]:
HOS2 = 0
hm = [1, 1, 2, 4, 10, 26, 76, 232, 764, 2620, 9496, 35696, 140152, 568504, 2390480, 10349536, 46206736, 211799312, 997313824, 4809701440]
cscscscs1 = comps2[len(comps2) - 10:len(comps2)]
cscscs1 = dict()
for i in cscscscs1:
    cscscs1[i[1]] = i[0]

for i in range(2 ** 9):
    preres = 0
    tst = []
    j1 = 0
    fl = True
    for j in compgra2.edges:
        if((i // (2 ** j1)) % 2 == 1):
            if(j[0] in tst or j[1] in tst):
                fl = False
                break
            tst.append(j[0])
            tst.append(j[1])
            preres += 1
        j1 += 1
    if(fl):
        pass
    else:
        preres = 1
        for i in tst:
            cscscs1[i] -= 1
        for i in cscscs1:
            preres *= hm[cscscs1[i]]
        for i in tst:
            cscscs1[i] += 1
        HOS2 += preres
print(HOS2 - HOS1)
print(HOS2)

221636040063236587123517685760
222281860605189510292431699968


In [306]:
vis1

{201102270008: True, 201102270009: True, 201102270010: True}

Тут я считаю expansion для ребер. Также учитываю что граф это дерево клик. Соответственно я перебираю клики которые
я добавляю в множество и для набора если его можно учитывать я считаю expansion для него и соответственно вывожу минимум

In [325]:
exp1 = 100000
cscscscs1 = comps1[len(comps1) - 10:len(comps1)]
for i in range(1, 2 ** 10):
    sm = 0
    res = 0
    tst = []
    for j in range(10):
        if((i // (2 ** j)) % 2 == 1):
            sm += cscscscs1[j][0]
            tst.append(cscscscs1[j][1])
    if(sm > len(workgra1.nodes) // 2):
        pass
    else:
        #print(tst)
        for j in range(10):
            if(cscscscs1[j][1] in tst):
                for k in compgra1.edges(cscscscs1[j][1]):
                    #print(k)
                    if(not k[1] in tst):
                        res += 1
        exp1 = min(exp1, res / sm)
exp1

0.03125

In [326]:
exp1 = 100000
cscscscs2 = comps2[len(comps2) - 10:len(comps2)]
for i in range(1, 2 ** 10):
    sm = 0
    res = 0
    tst = []
    for j in range(10):
        if((i // (2 ** j)) % 2 == 1):
            sm += cscscscs2[j][0]
            tst.append(cscscscs2[j][1])
    if(sm > len(workgra2.nodes) // 2):
        pass
    else:
        #print(tst)
        for j in range(10):
            if(cscscscs2[j][1] in tst):
                for k in compgra2.edges(cscscscs2[j][1]):
                    #print(k)
                    if(not k[1] in tst):
                        res += 1
        exp1 = min(exp1, res / sm)
exp1

0.029411764705882353

In [283]:
(57, 16) in compgra1.edges

True

Тут я считаю следующую перестановку(используется для GED)

In [264]:
def next_permutation(L):
    '''
    Permute the list L in-place to generate the next lexicographic permutation.
    Return True if such a permutation exists, else return False.
    '''
     
    n = len(L)
 
    #------------------------------------------------------------
 
    # Step 1: find rightmost position i such that L[i] < L[i+1]
    i = n - 2
    while i >= 0 and L[i] >= L[i+1]:
        i -= 1
     
    if i == -1:
        return False
 
    #------------------------------------------------------------
 
    # Step 2: find rightmost position j to the right of i such that L[j] > L[i]
    j = i + 1
    while j < n and L[j] > L[i]:
        j += 1
    j -= 1
     
    #------------------------------------------------------------
 
    # Step 3: swap L[i] and L[j]
    L[i], L[j] = L[j], L[i]
     
    #------------------------------------------------------------
 
    # Step 4: reverse everything to the right of i
    left = i + 1
    right = n - 1
 
    while left < right:
        L[left], L[right] = L[right], L[left]
        left += 1
        right -= 1
             
    return True

Тут я считаю Ядро графа как внутренне и внешне устойчивое множество вершин. Берется первая незакинутая вершина. Выкидываем
все смежные. Добавляем следующее итд. Усложненный алгоритм, который ищет наибольшее и наименьшее ядро других результатов по
размеру недал.(усложненный алгоритм пробует добавить сначала вершины, которые лежат на ребрах, соединяющих клики.

In [319]:
core1 = []
vis111 = dict()
for i in workgra1.nodes:
    if(i in vis111.keys()):
        pass
    else:
        core1.append(i)
        vis111[i] = True
        for j in workgra1.edges(i):
            vis111[j[1]] = True

print(core1)

core1 = []
vis111 = dict()
for i in workgra2.nodes:
    if(i in vis111.keys()):
        pass
    else:
        core1.append(i)
        vis111[i] = True
        for j in workgra2.edges(i):
            vis111[j[1]] = True

print(core1)

[200706130048, 200711280002, 200706070021, 200710010001, 200706080025, 200712260002, 200706010020, 200705270009, 200702130002, 200710310009]
[201102270009, 201102020011, 201111160068, 201111040012, 201112250025, 201105110003, 201110250027, 201103140009, 201101100019, 201101090007]


In [315]:
di = dict()
1 in di.keys()

False

## Схожесть на основе результатов

Схожесть я посчитал как взвешенную разность между двумя подсчитанными метриками(максимальной и минимальной для каждой метрики) деленную на максимум из этих двух метрик для каждого графа(то есть max(diam) - min(diam) / max(diam))
Чем ближе она к 0 тем графы более похожи

In [328]:
Gedf = 121
Di1f = diam1
Di2f = diam2
Ec1f = eccentricity1
Ec2f = eccentricity2
Pe1f = 14
Pe2f = 15
#girth не учитываю
Exp1f = 0.03125
Exp2f = 0.029411764705882353
Wi1f = 10947

Wi2f = 11597
#HOS1 HOS2
Cor1f = [200706130048, 200711280002, 200706070021, 200710010001, 200706080025, 200712260002, 200706010020, 200705270009, 200702130002, 200710310009]
Cor2f = [201102270009, 201102020011, 201111160068, 201111040012, 201112250025, 201105110003, 201110250027, 201103140009, 201101100019, 201101090007]

In [331]:
Dist = (diam2 - diam1) / (diam2) + (Pe2f - Pe1f) / (Pe2f) + (min(Ec1f) - min(Ec2f)) / max(Ec2f) + (Exp2f - Exp1f) / Exp2f + (Wi2f - Wi1f) / Wi2f  + (len(Cor2f) - len(Cor1f)) / len(Cor2f) + (HOS2 - HOS1) / HOS2

In [333]:
Dist / 7

0.15104431887791597